## Sztuczne sieci neuronowe - laboratorium 10

In [1]:
import torch
import time
import torch.nn as nn
import torchvision
from torchvision import models
from torchvision.datasets import ImageFolder
from torchvision.transforms import v2

In [2]:
# sprawdzenie, czy GPU jest widoczne
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda:0


## Transfer learning

Dzisiejsze zajęcia będą dotyczyły zagadnienia **transfer learning** - trenowania modeli polegającego na wykorzystaniu architektury i zestawu wag wytrenowanych wcześniej (np. przez kogoś innego - najczęściej badaczy z największych firm, na dużym zbiorze danych), celem wykorzystania "wiedzy" zgromadzonej w już wytrenowanym modelu i przeniesienia jej (stąd "transfer") do innego, zwykle węższego problemu (np. poprzez dotrenowanie na znacznie mniejszym zbiorze danych).

Fazy te nazywają się odpowiednio **pre-training** (tzw. modele pretrenowane, *pretrained models*) i **fine-tuning**.

Wiele z takich gotowych (pretrenowanych) modeli dostępnych jest w pakiecie `torchvision` - części PyTorcha związanej z przetwarzaniem obrazów.

#### Ćwiczenie
Uruchom poniższą komórkę, aby wypisać dostępne w `torchvision` modele. Nazwy modeli zaczynające się dużą literą oznaczają klasy implementujące poszczególne architektury sieci. Ich odpowiedniki pisane małymi literami to funkcje pozwalające zainicjalizować model (https://pytorch.org/vision/stable/models.html).

Funkcje te mają argument `pretrained` - gdy podamy wartość `True`, inicjalizujemy model pretrenowanymi wagami (dla `False` - losowymi).

Wczytaj po kolei wybrane modele (np. `resnet18`) do zmiennej. Sprawdź jej zawartość.

In [3]:
dir(models)

['AlexNet',
 'AlexNet_Weights',
 'ConvNeXt',
 'ConvNeXt_Base_Weights',
 'ConvNeXt_Large_Weights',
 'ConvNeXt_Small_Weights',
 'ConvNeXt_Tiny_Weights',
 'DenseNet',
 'DenseNet121_Weights',
 'DenseNet161_Weights',
 'DenseNet169_Weights',
 'DenseNet201_Weights',
 'EfficientNet',
 'EfficientNet_B0_Weights',
 'EfficientNet_B1_Weights',
 'EfficientNet_B2_Weights',
 'EfficientNet_B3_Weights',
 'EfficientNet_B4_Weights',
 'EfficientNet_B5_Weights',
 'EfficientNet_B6_Weights',
 'EfficientNet_B7_Weights',
 'EfficientNet_V2_L_Weights',
 'EfficientNet_V2_M_Weights',
 'EfficientNet_V2_S_Weights',
 'GoogLeNet',
 'GoogLeNetOutputs',
 'GoogLeNet_Weights',
 'Inception3',
 'InceptionOutputs',
 'Inception_V3_Weights',
 'MNASNet',
 'MNASNet0_5_Weights',
 'MNASNet0_75_Weights',
 'MNASNet1_0_Weights',
 'MNASNet1_3_Weights',
 'MaxVit',
 'MaxVit_T_Weights',
 'MobileNetV2',
 'MobileNetV3',
 'MobileNet_V2_Weights',
 'MobileNet_V3_Large_Weights',
 'MobileNet_V3_Small_Weights',
 'RegNet',
 'RegNet_X_16GF_Weights'

In [4]:
model = models.resnet18()
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### Klasyfikacja binarna - przygotowanie danych

Będziemy trenować model do klasyfikacji binarnej zdjęć pszczół i mrówek z wykorzystaniem transfer learningu.

Zbiór ten jest dostępny do pobrania tutaj: https://download.pytorch.org/tutorial/hymenoptera_data.zip  
(*hymenoptera* - *błonoskrzydłe* https://pl.wikipedia.org/wiki/B%C5%82onkoskrzyd%C5%82e)

Zbiór ten należy rozpakować do katalogu `common/data` (w razie gdyby go jeszcze tam nie było).

In [5]:
import pathlib

DATA_PATH = pathlib.Path("data/hymenoptera_data")

#### Ćwiczenie

Wczytaj zbiór zdjęć (dwa podbiory - train i val) wykorzystując klasę **ImageFolder** dataset dostępną w PyTorch (https://pytorch.org/vision/stable/datasets.html)

Sprawdź rozmiar obu podzbiorów.  
Sprawdź wymiary kilku wybranych zdjęć w zbiorze (używając możliwości klasy `Dataset`, nie przeglądając obrazki w katalogu).

Sprawdź zawartość atrybutów klasy `ImageFolder` (https://pytorch.org/vision/stable/_modules/torchvision/datasets/folder.html#ImageFolder)

In [6]:
transform = v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])
train_path = DATA_PATH / "train"
val_path = DATA_PATH / "val"
train_dataset = ImageFolder(train_path, transform=transform)
val_dataset = ImageFolder(val_path, transform=transform)

In [7]:
print(len(train_dataset))
print(len(val_dataset))
train_image, train_label = train_dataset[0]
print(train_image.size())
val_image, val_label = val_dataset[0]
print(val_image.size())

244
153
torch.Size([3, 512, 768])
torch.Size([3, 375, 500])


In [8]:
dir(ImageFolder)

['__add__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_format_transform_repr',
 '_repr_indent',
 'extra_repr',
 'find_classes',
 'make_dataset']

#### Ćwiczenie

Domyślnie `ImageFolder` dataset przechowuje obrazki jako `PIL.Image`. Należy je przekształcić do tensorów, aby ich użyć w treningu modeli.

Odszukaj odpowiednią funkcję z `torchvision.transforms` (https://pytorch.org/vision/stable/transforms.html) i ponownie wczytaj zbiory danych z jej wykorzystaniem.

In [9]:
transform = v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])
train_path = DATA_PATH / "train"
val_path = DATA_PATH / "val"
train_dataset = ImageFolder(train_path, transform=transform)
val_dataset = ImageFolder(val_path, transform=transform)

#### Ćwiczenie
W kolejnym kroku należy znormalizować wejściowe obrazki. Ponownie odszukaj odpowiednią transformację w `torchvision.transforms` i zbuduj listy transformacji `train_transforms` i `valid_transforms` z użyciem `transforms.Compose`. Wykorzystaj odpowiednie informacje z poniższej komórki.

In [10]:
# średnie i odchylenia standardowe dla kanałów RGB dla zbioru uczącego ImageNet
# ciekawostka: https://github.com/pytorch/vision/issues/1439
IMAGENET_MEANS = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

In [11]:
train_transforms = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=IMAGENET_MEANS, std=IMAGENET_STD)
])

val_transforms = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=IMAGENET_MEANS, std=IMAGENET_STD)
])

#### Ćwiczenie

Należy także doprowadzić oryginalne obrazki do odpowiednich wymiarów (224 na 224 piksele).

W przypadku sieci (pre)trenowanych na danych ImageNet przyjęło się robić to dwukrokowo:
- "resize" obrazka, aby krótszy wymiar miał długość 256
- przycięcie ("crop") obrazka do jego środkowej części 224x224

Rozszerz listę transformacji **podczas walidacji** zgodnie z powyższym opisem, wykorzystując odpowiednie funkcje z https://pytorch.org/vision/stable/transforms.html. Transformacjami treningowymi zajmiemy się w następnym ćwiczeniu.

In [12]:
IMAGENET_IMG_SIZE = 224
IMAGENET_RESIZE = 256

In [13]:
val_transforms = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Resize(IMAGENET_RESIZE),
    v2.CenterCrop(IMAGENET_IMG_SIZE),
    v2.Normalize(mean=IMAGENET_MEANS, std=IMAGENET_STD)
])

#### Ćwiczenie

Podczas treningu warto - zwłaszcza w przypadku posiadania niewielkiego zbioru danych - zastosować tzw. augmentację danych (więcej na kolejnych zajęciach).

Zamiast "sztywnego" resize'owania obrazka i przycinania go względem środka, w czasie treningu:
- dokonaj "resize" do losowej skali, a następnie przytnij do (losowego) fragmentu 224x224
- dodatkowo losowo (domyślnie: prawdopodobieństwo 50%) przerzuć obrazek względem osi pionowej

Znajdź odpowiednie funkcje w https://pytorch.org/vision/stable/transforms.html.
Stwórz w ten sposób listę transformacji `train_transforms`.

Wczytaj ponownie zbiór uczący i walidacyjny, podając odpowiednie listy transformacji do `ImageFolder`.

In [14]:
train_transforms = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.RandomResizedCrop(IMAGENET_IMG_SIZE),
    v2.RandomVerticalFlip(),
    v2.Normalize(mean=IMAGENET_MEANS, std=IMAGENET_STD)
])

In [15]:
train_dataset = ImageFolder(train_path, transform=train_transforms)
val_dataset = ImageFolder(val_path, transform=val_transforms)

### Transfer learning

Fine-tuningu modeli można dokonać na dwa główne sposoby:
- dotrenować (optymalizować) wszystkie parametry (we wszystkich warstwach) pretrenowanego modelu
- "zamrozić" pretrenowaną część modelu i dotrenować

Na początek zajmiemy się pierwszym z wymienionych sposobów transfer learningu.

#### Ćwiczenie

Załaduj pretrenowaną sieć `resnet18` do zmiennej `model` i dostosuj ją do rozważanego problemu (co musisz zrobić?).
Następnie uruchom trening modelu.

In [16]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    start_time = time.time()

    model.train()
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)
            labels = labels.to(device=device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_train += loss.item()

        epoch_loss = loss_train / len(train_loader)
        if epoch == 1 or epoch % 5 == 0:
            print(f"Epoch {epoch}, Training loss {epoch_loss}")

    time_elapsed = time.time() - start_time
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))

In [17]:
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT).to(device)

In [18]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

# SGD with momentum
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 25,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader
)

Epoch 1, Training loss 5.311984598636627
Epoch 5, Training loss 0.36217445135116577
Epoch 10, Training loss 0.2655116282403469
Epoch 15, Training loss 0.17122965026646852
Epoch 20, Training loss 0.11138878762722015
Epoch 25, Training loss 0.04979503434151411
Training complete in 0m 40s


#### Ćwiczenie

Sprawdź jakość wytrenowanego modelu uruchamiając poniższe komórki.

In [19]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=False)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False)

In [20]:
def validate(model, train_loader, val_loader):
    model.eval()
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad():
            for imgs, labels in loader:
                imgs = imgs.to(device)
                labels = labels.to(device)
                outputs = model(imgs)
                preds = torch.argmax(outputs, dim=1)
                total += labels.shape[0]
                correct += int((preds == labels).sum())

        print(f"{name} accuracy: {correct/total}")

In [21]:
validate(model, train_loader, val_loader)

train accuracy: 0.9795081967213115
val accuracy: 0.8562091503267973


#### Ćwiczenie

Jeszcze raz załaduj i przygotuj model `resnet18` (np. do zmiennej `model_frozen`, tym razem "zamrażając" wszystkie pretrenowane warstwy modelu.
Wytrenuj model i sprawdź jego dokładność.

In [22]:
model_frozen = models.resnet18(weights=models.ResNet18_Weights.DEFAULT).to(device)
for param in model_frozen.parameters():
    param.requires_grad = False
for param in model_frozen.fc.parameters():
    param.requires_grad = True

In [23]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

# SGD with momentum
optimizer = torch.optim.SGD(model_frozen.parameters(), lr=1e-2, momentum=0.9)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 25,
    optimizer = optimizer,
    model = model_frozen,
    loss_fn = loss_fn,
    train_loader = train_loader
)

Epoch 1, Training loss 5.883200943470001
Epoch 5, Training loss 0.33941885456442833
Epoch 10, Training loss 0.2079898789525032
Epoch 15, Training loss 0.13808961398899555
Epoch 20, Training loss 0.11620101425796747
Epoch 25, Training loss 0.1448836624622345
Training complete in 0m 27s


In [24]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=False)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False)

In [25]:
validate(model_frozen, train_loader, val_loader)

train accuracy: 0.9631147540983607
val accuracy: 0.9411764705882353


#### Ćwiczenie

Porównaj powyższe wyniki z uzyskanymi dla modelu `Net` stworzonego na wcześniejszych zajęciach (lekko zmodyfikowane wymiary dla warstw gęstych - inny rozmiar obrazków wejściowych).

In [26]:
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(8 * 56 * 56, 32)
        self.fc2 = nn.Linear(32, 2)

    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = out.view(-1, 8 * 56 * 56)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out

In [27]:
net_model = Net()
net_model = net_model.to(device)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

# SGD with momentum
optimizer = torch.optim.SGD(net_model.parameters(), lr=1e-2, momentum=0.9)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 25,
    optimizer = optimizer,
    model = net_model,
    loss_fn = loss_fn,
    train_loader = train_loader
)

Epoch 1, Training loss 0.6892561465501785
Epoch 5, Training loss 0.6995721310377121
Epoch 10, Training loss 0.6391260623931885
Epoch 15, Training loss 0.606040894985199
Epoch 20, Training loss 0.5905006229877472
Epoch 25, Training loss 0.5931582301855087
Training complete in 0m 24s


In [28]:
validate(net_model, train_loader, val_loader)

train accuracy: 0.6598360655737705
val accuracy: 0.6339869281045751


#### Wnioski
Dzięki zajęciom zrozumiałem czym jest transfer learning. Słyszałem już o nim wcześniej w internecie lecz nie wiedziałem, że tak się nazywa. Myślę, iż pozwala na zaoszczędzenie czasu na treningu modelu. Sam planowałem z niego skorzystać w mojej pracy inżynierskiej.